# Databricks Data Preparation in ML - Notebook 01
## Exploring Data in Data Lakehouse for ML

**Part of the Databricks Data Preparation in ML Training Series**

---

## Objectives

This notebook covers fundamental data exploration techniques in Databricks Data Lakehouse architecture, focusing on:

- **Data Lakehouse Architecture** - Understanding Medallion Architecture layers
- **Unity Catalog Fundamentals** - Data governance and organization for ML
- **Delta Lake Basics** - ACID transactions and time travel capabilities
- **Data Exploration Techniques** - display() vs show() and advanced profiling
- **Performance Optimization** - Caching, partitioning, and query optimization
- **ML-Specific Data Analysis** - Feature correlation and target variable analysis

## Duration: ~30 minutes
## Level: Fundamental

---

## Why Data Lakehouse for ML?

**Data Lakehouse** combines the best of data lakes and data warehouses:
- **Unified Storage** - All data types in one platform
- **ACID Transactions** - Enterprise-grade reliability
- **Schema Evolution** - Flexible data structure changes
- **Performance** - Optimized for both analytics and ML workloads
- **Governance** - Built-in security and data lineage

### ML Benefits:
- **Feature Store Integration** - Centralized feature management
- **Experiment Tracking** - MLflow native integration
- **Data Versioning** - Time travel for reproducible ML
- **Real-time Serving** - Low-latency feature serving

# Unity Catalog Fundamentals

## Theory

**Unity Catalog** is Databricks' unified governance solution for data and AI assets. It provides a single place to administer data access policies across all Databricks workspaces in your organization.

### Unity Catalog Hierarchy:

```
Metastore
├── Catalog
│   ├── Schema (Database)
│   │   ├── Tables
│   │   ├── Views
│   │   ├── Functions
│   │   └── Volumes
│   └── Models (MLflow)
└── Connections (External Systems)
```

### Key Benefits for ML:

#### **Data Governance**
- **Centralized Access Control** - Fine-grained permissions at table/column level
- **Data Lineage** - Track data flow from source to ML models
- **Audit Logging** - Complete audit trail for compliance
- **Data Discovery** - Search and discover datasets across organization

#### **ML Lifecycle Integration**
- **Feature Store** - Register and discover features across teams
- **Model Registry** - Version and govern ML models
- **Experiment Tracking** - Link experiments to data lineage
- **Data Versioning** - Time travel for reproducible ML

#### **Cross-Workspace Collaboration**
- **Shared Data Assets** - Access data across multiple workspaces
- **Consistent Naming** - Standardized catalog structure
- **Team Collaboration** - Share datasets and models safely

## Environment Setup

In [0]:
# Essential imports for Databricks ML data exploration
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnan, isnull, when, desc, asc
from pyspark.sql.functions import mean, stddev, min as spark_min, max as spark_max, percentile_approx
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as F

In [0]:
# Verify Spark session and Databricks environment
spark.sql("SELECT 'Databricks environment ready!' as status").show()
spark.sql("SELECT current_catalog() as current_catalog, current_schema() as current_schema").show()

#Creating Sample Data - Medallion Architecture Demo

## Practical Implementation

Let's create sample datasets that demonstrate the Medallion Architecture layers. We'll simulate a customer analytics use case for ML model training.

In [0]:
pip install faker

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Zakładamy, że sesja Spark już istnieje
# spark = SparkSession.builder.getOrCreate()

# Dane z warstwy Bronze (108 wierszy)
bronze_data = [
    (1, "John Doe", "25", "john.doe@email.com", "50000", "2023-01-15", "CRM_SYSTEM"),
    (2, "Jane Smith", "", "jane@company.com", "75000", "2023-02-20", "CRM_SYSTEM"),
    (3, "Bob Johnson", "35", "", "60000", "2023-03-10", "HR_SYSTEM"),
    (4, "Alice Brown", "28", "alice.brown@email.com", "", "2023-04-05", "CRM_SYSTEM"),
    (5, None, "45", "unknown@test.com", "85000", "2023-05-12", "HR_SYSTEM"),
    (6, "Charlie Wilson", "abc", "charlie@email.com", "70000", "2023-06-18", "CRM_SYSTEM"),
    (7, "Diana Davis", "31", "diana.davis@email.com", "90000", "2023-07-22", "CRM_SYSTEM"),
    (1, "John Doe", "25", "john.doe@email.com", "50000", "2023-01-15", "CRM_SYSTEM")  # Duplicate
]

# --- Generowanie kolejnych 100 wierszy z losowymi brakami danych ---
import random
from faker import Faker

fake = Faker()
sources = ["CRM_SYSTEM", "HR_SYSTEM", "ERP_SYSTEM", "WEB_PORTAL", "MOBILE_APP"]

for i in range(8, 1000):
    name = fake.name() if random.random() > 0.05 else None
    age = str(random.randint(18, 65)) if random.random() > 0.1 else ""
    if random.random() < 0.05:
        age = "xyz"
    email = fake.email() if random.random() > 0.1 else ""
    salary = str(random.randint(30000, 120000)) if random.random() > 0.1 else ""
    signup_date = fake.date_between(start_date="-2y", end_date="today").strftime("%Y-%m-%d")
    source = random.choice(sources)

    bronze_data.append((i, name, age, email, salary, signup_date, source))

# --- Tworzenie DataFrame ---
bronze_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("full_name", StringType(), True),
    StructField("age", StringType(), True),  # Raw as string (may contain invalid data)
    StructField("email", StringType(), True),
    StructField("salary", StringType(), True),  # Raw as string (may contain invalid data)
    StructField("registration_date", StringType(), True),
    StructField("source_system", StringType(), True)
])

df_bronze = spark.createDataFrame(bronze_data, schema=bronze_schema)

# --- Wyświetlenie danych ---
display(df_bronze)

In [0]:
df_bronze.write.format("csv").mode("overwrite").option("header", "true").save("/FileStore/tables/bronze_data.csv")

In [0]:
# Display Bronze layer data - Raw data as ingested
display(df_bronze)

#Data Exploration Techniques

## Display vs Show - Practical Differences

Understanding when to use `display()` vs `show()` is crucial for effective data exploration in Databricks.

In [0]:
# show() - Standard Spark function for tabular output
# Best for: debugging, programmatic output, truncated views
df_bronze.show(5, truncate=False)

In [0]:
# display() - Databricks-specific function with rich features
# Best for: interactive exploration, visualizations, data profiling
# Features: sorting, filtering, plotting, statistics
display(df_bronze)

## Advanced Data Profiling

Beyond basic `describe()`, we can perform comprehensive data profiling to understand data quality and characteristics.

In [0]:
# Comprehensive data profiling
total_records = df_bronze.count()
total_columns = len(df_bronze.columns)

# Create profiling summary
profiling_data = [
    ("Total Records", total_records),
    ("Total Columns", total_columns),
    ("Source Systems", df_bronze.select("source_system").distinct().count())
]

profiling_df = spark.createDataFrame(profiling_data, ["Metric", "Value"])
display(profiling_df)

In [0]:
# Display basic statistics for all columns in the Bronze DataFrame
display(df_bronze.describe())

In [0]:
# Display the raw Bronze layer data for interactive exploration
display(df_bronze)

In [0]:
# Null value analysis per column
null_analysis = df_bronze.select([
    count(when(col(c).isNull(), c)).alias(f"{c}_nulls") for c in df_bronze.columns
])

display(null_analysis)

In [0]:
df_bronze.count()

In [0]:
dr_dedup_bronze = df_bronze.dropDuplicates()

In [0]:
dr_dedup_bronze.count()


## Spark SQL Integration for ML

Spark SQL provides powerful analytical capabilities that are essential for ML data exploration.

In [0]:
# Create temporary view for SQL analysis
df_bronze.createOrReplaceTempView("bronze_customers")

In [0]:
%sql

SELECT  * FROM bronze_customers

In [0]:
%sql
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT customer_id) as unique_customers,
    SUM(CASE WHEN full_name IS NULL THEN 1 ELSE 0 END) as missing_names,
    SUM(CASE WHEN email = '' OR email IS NULL THEN 1 ELSE 0 END) as missing_emails,
    SUM(CASE WHEN salary = '' OR salary IS NULL THEN 1 ELSE 0 END) as missing_salaries
FROM bronze_customers 
GROUP BY source_system
ORDER BY source_system

# Delta Lake Fundamentals for ML

## Theory

**Delta Lake** is an open-source storage layer that brings ACID transactions to Apache Spark and big data workloads. It's the foundation of the Databricks Lakehouse.

### 🎯 Key Benefits for ML:

#### **ACID Transactions**
- **Atomicity** - All-or-nothing operations
- **Consistency** - Data integrity maintained
- **Isolation** - Concurrent operations don't interfere
- **Durability** - Changes are permanent

#### **Time Travel**
- **Data Versioning** - Every write creates a new version
- **Reproducible ML** - Access exact data used for training
- **Experiment Tracking** - Link models to specific data versions
- **Rollback Capability** - Revert to previous data states

#### **Schema Evolution**
- **Schema Enforcement** - Prevent data quality issues
- **Schema Evolution** - Add/modify columns safely
- **Compatibility** - Maintain backward compatibility

In [0]:
# Create Delta table from Bronze data
# Note: In production, you would write to a proper table location
df_bronze.write.format("delta").mode("overwrite").saveAsTable("bronze_customers_delta")

In [0]:
# Read from Delta table
df_delta = spark.table("bronze_customers_delta")
display(df_delta)

In [0]:
# Delta table history - essential for ML reproducibility
display(spark.sql("DESCRIBE HISTORY bronze_customers_delta"))

In [0]:
%sql
insert into bronze_customers_delta
select * from bronze_customers_delta limit 10

In [0]:
%sql
--Time travel example: query the bronze_customers_delta table as of version 1
SELECT * FROM bronze_customers_delta VERSION AS OF 1


# Performance Optimization for ML Workloads

## Caching Strategies

Caching is crucial for iterative ML workflows where the same dataset is accessed multiple times.

In [0]:
# Cache the DataFrame for repeated access
df_bronze_cached = df_bronze.cache()

# Force caching by performing an action
df_bronze_cached.count()

In [0]:
# Column pruning - select only needed columns for ML features
ml_features = df_bronze_cached.select("customer_id", "age", "email", "salary")
display(ml_features)


This cell can be used to explain the difference between shallow and deep clone in Delta Lake:

- **Shallow Clone**: Creates a new Delta table that references the same data files as the source table. Only metadata is copied, so it's fast and storage-efficient. Changes to the clone do not affect the source, but the underlying data is shared until modified.
- **Deep Clone**: Copies both the metadata and the data files to a new location. The clone is fully independent of the source table, and changes to either do not affect the other. Useful for backups or migrating data.

Use this cell to document these concepts or provide examples.

In [0]:
# Shallow clone of a table (metadata only, no data copy)
spark.sql("""
CREATE OR REPLACE TABLE bronze_customers_delta_shallow_clone
SHALLOW CLONE bronze_customers_delta
""")

# Deep clone of a table (full data copy)
spark.sql("""
CREATE OR REPLACE TABLE bronze_customers_delta_deep_clone
DEEP CLONE bronze_customers_delta
""")

# ML-Specific Data Analysis

## Creating Silver Layer with Cleaned Data

Transform Bronze data into Silver layer with proper data types and cleaning for ML workflows.

In [0]:
# Silver layer transformation - clean data with proper types
df_silver = df_bronze.select(
    col("customer_id"),
    col("full_name"),
    # Convert age to integer, handle invalid values
    when(col("age").rlike("^[0-9]+$"), col("age").cast("int")).otherwise(None).alias("age_clean"),
    col("email"),
    # Convert salary to double, handle invalid values  
    when(col("salary").rlike("^[0-9]+$"), col("salary").cast("double")).otherwise(None).alias("salary_clean"),
    col("registration_date"),
    col("source_system")
).filter(col("customer_id").isNotNull())  # Remove records without customer_id

# Remove duplicates based on customer_id
df_silver = df_silver.dropDuplicates(["customer_id"])

display(df_silver)

In [0]:
# Feature correlation analysis for ML
# Select only numeric columns for correlation
numeric_features = df_silver.select("age_clean", "salary_clean").filter(
    col("age_clean").isNotNull() & col("salary_clean").isNotNull()
)

# Basic statistics for numeric features
display(numeric_features.describe())

In [0]:
# Data distribution analysis - key for ML feature engineering
age_stats = df_silver.select(
    mean("age_clean").alias("mean_age"),
    stddev("age_clean").alias("stddev_age"),
    spark_min("age_clean").alias("min_age"),
    spark_max("age_clean").alias("max_age"),
    percentile_approx("age_clean", 0.25).alias("q1_age"),
    percentile_approx("age_clean", 0.75).alias("q3_age")
)

display(age_stats)

# Data Visualization for ML

## Built-in Databricks Visualizations

Databricks provides powerful built-in visualization capabilities that are essential for ML data exploration and feature analysis.

In [0]:
# Create a cleaned dataset for visualization
# Convert string columns to appropriate types for visualization
df_viz = df_bronze.filter(col("customer_id").isNotNull()) \
    .withColumn("age_numeric", 
                when(col("age").rlike("^[0-9]+$"), col("age").cast("int"))
                .otherwise(None)) \
    .withColumn("salary_numeric", 
                when(col("salary").rlike("^[0-9]+$"), col("salary").cast("double"))
                .otherwise(None)) \
    .filter(col("age_numeric").isNotNull() & col("salary_numeric").isNotNull())


## Custom Visualization in Databricks Using Native Options

The following code demonstrates how to create custom visualizations directly within Databricks using its built-in visualization tools.

In [0]:
# Feature correlation: Age vs Salary - essential for ML feature engineering
display(df_viz.select("age_numeric", "salary_numeric"))

In [0]:
# Data source distribution - understanding data lineage
source_distribution = df_bronze.groupBy("source_system").count().orderBy(desc("count"))
display(source_distribution)

## Advanced Visualizations with Python Libraries

For more sophisticated ML visualizations, we can integrate matplotlib and seaborn with Databricks.

In [0]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [0]:
# Correlation heatmap for ML feature analysis
# Convert to Pandas for advanced visualization
pandas_df = df_viz.select("age_numeric", "salary_numeric").toPandas()

In [0]:
pandas_df.display()

This code generates a correlation heatmap for the features in a pandas DataFrame. It uses matplotlib to set the figure size and seaborn to plot the correlation matrix, displaying correlation coefficients with color gradients and annotations. The heatmap helps visualize relationships between features, which is useful for machine learning analysis.

In [0]:
# Create correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(pandas_df.corr(), annot=True, cmap='coolwarm', center=0, fmt='.3f')
plt.title('Feature Correlation Matrix for ML')
plt.tight_layout()
plt.show()

This code creates side-by-side histograms to visualize the distributions of the `age_numeric` and `salary_numeric` features from a pandas DataFrame. It helps identify the spread, central tendency, and potential outliers in these features, which is useful for ML feature engineering.

In [0]:
display(pandas_df)

In [0]:
# Distribution analysis for ML feature engineering
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Age distribution
axes[0].hist(pandas_df['age_numeric'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Age Distribution')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')  

# Salary distribution
axes[1].hist(pandas_df['salary_numeric'], bins=10, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1].set_title('Salary Distribution')
axes[1].set_xlabel('Salary')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [0]:
df_viz.createOrReplaceTempView("viz_view")

In [0]:
%sql

select * from viz_view

In [0]:
%sql
with hist_age_stand as (
  select 
    case 
      when age >= 18 and age < 20 then '18-19'
      when age >= 20 and age < 25 then '20-24'
      when age >= 25 and age < 30 then '25-29'
      when age >= 30 and age < 35 then '30-34'
      when age >= 35 and age < 40 then '35-39'
      when age >= 40 and age < 45 then '40-44'
      when age >= 45 and age < 50 then '45-49'
      when age >= 50 and age < 55 then '50-54'
      when age >= 55 and age < 60 then '55-59'
      when age >= 60 and age < 65 then '60-64'
      when age >= 65 and age < 70 then '65-69'
      when age >= 70 and age < 75 then '70-74'
      when age >= 75 and age < 80 then '75-79'
      else '80+' 
    end as age_standardization,
    age 
  from viz_view
)
select age_standardization, count(age) 
from hist_age_stand
group by age_standardization
order by age_standardization